실습N #2: 사용자가 입력한 지역과 질문에 맞는 지역 음식점을 검색해서 추천하는 어플리케이션을 작성하시오.
예) 판교에 있는 괜찮은 이탈리안 식당을 몇개 추천해줄래?

In [ ]:
import requests
import openai
import os

# ==== 사용자 입력 ====
region = input("지역을 입력하세요 (예: 강남역, 부산 해운대 등): ")
food_type = input("음식 종류를 입력하세요 (예: 한식, 일식, 고기 등): ")

# ==== 설정 ====
KAKAO_API_KEY = "d451a44400e161bb68d466933d9d"
#OPENAI_API_KEY = "YOUR_OPENAI_API_KEY"
openai.api_key = os.getenv("OPENAI_API_KEY")

# ==== 1. 카카오 API로 음식점 검색 ====
def search_places(region, food_type):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    query = f"{region} {food_type} 맛집"
    params = {"query": query, "size": 10}

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    places = []
    for doc in data.get("documents", []):
        places.append({
            "name": doc["place_name"],
            "address": doc["road_address_name"],
            "phone": doc["phone"],
            "category": doc["category_name"]
        })
    return places

# ==== 2. GPT에게 추천 설명 요청 ====
def recommend_places_with_llm(places, region, food_type):
    prompt = f"""
사용자가 {region} 지역에서 {food_type}을(를) 찾고 있습니다.
다음은 해당 지역의 추천 음식점 목록입니다:

{places}

이 중에서 추천할 만한 음식점을 3개 뽑아 설명과 함께 추천해주세요.
설명은 맛, 분위기, 위치 등을 포함해서 작성해주세요.
"""

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",  # 또는 "gpt-3.5-turbo"
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )

    return response.choices[0].message.content

# ==== 3. 실행 ====
places = search_places(region, food_type)

if not places:
    print("검색된 음식점이 없습니다. 다른 지역이나 음식 종류를 입력해주세요.")
else:
    llm_response = recommend_places_with_llm(places, region, food_type)
    print("\n📌 추천 결과:\n")
    print(llm_response)

검색된 음식점이 없습니다. 다른 지역이나 음식 종류를 입력해주세요.
